# Simple fit and fit result printout

This notebook shows how to perform a simple fit on the workspace produced earlier.

It also shows various ways of printing and saving the results of the fit.

Open the workspace file and extract the workspace object:

In [ ]:
f = new TFile("../ws/ICTPws_test_combined_ICTPws_test_model.root");
w = dynamic_cast<RooWorkspace*>(f->Get("combined"));

Extract the ModelConfig from the workspace:

In [ ]:
mc = dynamic_cast<RooStats::ModelConfig*>(w->obj("ModelConfig"));

Get the data we want to fit:

In [ ]:
dataset = w->data("obsData");

Now perform a S+B fit:
* set the initial value of the parameter of interest (POI)
* make sure it is not fixed, and it is free to float in the fit
* call the "fitTo" method

In [ ]:
w->var("mu_ttH")->setVal(0);
w->var("mu_ttH")->setConstant(kFALSE);
w->pdf("simPdf")->fitTo(*dataset);

From the output above, we can find the fitted values of all the parameters (of interest and nuisance) and their uncertainties.

Now, let's try to extract and print all and only the information we want, so that we could re-use it later:
* the fitted value of the POI, with its error
* the fitted values of the NPs, with their errors (what we call the "pulls" and "constraints" of the NPs)

Let's extract the POI and print the fitted value and uncertainties (separately the up and down)

In [ ]:
RooRealVar *poi = (RooRealVar*)mc->GetParametersOfInterest()->first();
double mu_hat = poi->getVal();
double mu_hat_err_up = poi->getErrorHi();
double mu_hat_err_down = poi->getErrorLo();

cout << setw(25);
cout << "POI =";
cout << Form(" %+.3f",mu_hat);
cout << Form(" %+.3f",mu_hat_err_up);
cout << " /";
cout << Form(" %+.3f",mu_hat_err_down);
cout << endl;

And now for all the NPs

In [ ]:
// loop on list of nuisance parameters
for(auto np_tmp : *mc->GetNuisanceParameters()){
    RooRealVar* np = (RooRealVar*)np_tmp;
    
    string np_name = np->GetName();
    
    double np_value = np->getVal();
    double np_err_up = np->getErrorHi();
    double np_err_down = np->getErrorLo();
    
    cout << setw(25);
    cout << np_name << " =";
    cout << Form(" %+.3f",np_value);
    cout << Form(" %+.3f",np_err_up);
    cout << " /";
    cout << Form(" %+.3f",np_err_down);
    cout << endl;
}

An alternative way (and more elegant) to store the fit results (with more information, to use later) is to use the `FitResult`

In [ ]:
RooFitResult *r = w->pdf("simPdf")->fitTo(*dataset, RooFit::Save());
r->Print();

Or, as before, we might want to extract only some of the information:

In [ ]:
for(auto par_tmp : r->floatParsFinal()){
    RooRealVar* par = (RooRealVar*)par_tmp;
    
    string par_name = par->GetName();
    
    double par_value = par->getVal();
    double par_err_up = par->getErrorHi();
    double par_err_down = par->getErrorLo();
    
    cout << setw(25);
    cout << par_name << " =";
    cout << Form(" %+.3f",par_value);
    cout << Form(" %+.3f",par_err_up);
    cout << " /";
    cout << Form(" %+.3f",par_err_down);
    cout << endl;
}

## Excercise

Let's try to produce the results of a background-only fit, and compare the output with that of the nominal S+B fit.
Hint: to perform a B-only fit, the easiest solution is to fix the POI (to what value?)